In [1]:
import os
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np

In [4]:
import pandas as pd
import functions

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor

In [5]:
# Whole_text.to_pickle('whole_text.pkl')
whole_text = pd.read_pickle('whole_text.pkl')
error_text = whole_text.loc[whole_text.SAME==False].drop_duplicates(keep=False)
# np.save('lexicon.npy', {'GROUP_1':lexicon1, 'GROUP_2':lexicon2,'GROUP_3':lexicon3,'GROUP_4':lexicon4,'GROUP_5':lexicon5})
lexicon = np.load('lexicon.npy').item()
# np.save('dictionary.npy', Dictionary) 
dictionary = np.load('dictionary.npy').item()
# np.save('dictionary_exact.npy', Three_gram_dictionary)
dictionary_exact = np.load('dictionary_exact.npy').item()
# np.save('dictionary_relaxed.npy', Three_gram_dictionary_x) 
dictionary_relaxed = np.load('dictionary_relaxed.npy').item()

In [8]:
error_text.head(5)

,prev_4,prev_3,prev_2,prev_1,WORD_OCR,next_1,next_2,next_3,next_4,WORD_TRUE,SAME,GROUP
0,*,*,*,*,xansm,u,ma,rah,by,exhibit,False,GROUP_1
3,*,xansm,u,ma,rah,by,willlam,j,drlver,rch,False,GROUP_1
7,rah,by,willlam,j,drlver,congressional,action,nvolvng,portlons,driver,False,GROUP_1
10,j,drlver,congressional,action,nvolvng,portlons,of,t,orelgn,involving,False,GROUP_1
14,nvolvng,portlons,of,t,orelgn,trade,and,investment,act,foreign,False,GROUP_1


In [332]:
We = list(error_text['WORD_OCR'])
Truth = list(error_text['WORD_TRUE'])
Group = list(error_text['GROUP'])
dataset = pd.DataFrame()
for i in range(len(We)):
    data_piece = pd.DataFrame()
    for Threshold in range(len(dataset)):
        candidates = p4.candidate_search(dictionary, We[i], Threshold)
        if len(candidates) >= 10:
            break

    dist_score = p4.distance_score(candidates, We, Threshold)
    simi_score = p4.similarity_score(candidates, We, a1, a2, a3, a4)
    popu_score = p4.popularity_score(candidates)
    exis_score = p4.existance_score(candidates, lexicon[Group[i]])
    exat_score = p4.exact_popularity_score(candidates, We[i], three_gram(error_text.iloc[i]), dictionary_exact)
    rela_score = p4.relaxed_popularity_score(candidates, We[i], three_gram(error_text.iloc[i]), dictionary_relaxed)
            
    data_piece["We"] = [We[i]]*len(candidates)
    data_piece["Wc"] = candidates.keys()
    data_piece["x1"] = dist_score.values()
    data_piece["x2"] = simi_score.values()
    data_piece["x3"] = popu_score.values()
    data_piece["x4"] = exis_score.values()
    data_piece["x5"] = exat_score.values()
    data_piece["x6"] = rela_score.values()
    
    label_list = []
    for j in range(len(candidates)):
        if data_piece.iloc[j,1] == Truth[i]:
            label_list.append(1)
        else:
            label_list.append(0)
    data_piece["label"] = label_list
    
    dataset = dataset.append(data_piece,ignore_index=True)

KeyboardInterrupt: 

In [333]:
# dataset['Weight'] = dataset['label']*len(dataset) / sum(dataset['label'])
X_train, X_test, y_train, y_test= train_test_split(dataset.iloc[:,2:7], dataset.iloc[:,[0,1,8]], test_size=0.25)

In [334]:
regressor = AdaBoostRegressor()
regressor.fit(X_train,y_train['label'])

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)

In [335]:
#pd.Index(y_test).get_loc(1)

In [336]:
prediction = regressor.predict(X_test).tolist()

In [337]:
sum(prediction)

13.252417794970974

In [338]:
sum(dataset.label)

48

In [339]:
result = y_test

In [340]:
result['prediction']=prediction

In [341]:
result.groupby("We").max()

,Wc,label,prediction
We,,,
admlnlstratlons,ministrators,0,0.045455
aflcid,avoid,0,0.035461
called,hailed,0,0.454545
chalman,waxman,0,0.000000
cm,tem,0,0.035461
cmng,sng,0,0.035461
conslderatlon,sideration,0,0.035461
contrlbute,contributing,0,0.035461
dlstrlbutlon,redistribution,0,0.000000


In [349]:
result.loc[result.groupby('We')['prediction'].idxmax()]

,We,Wc,label,prediction
239,admlnlstratlons,administration,0,0.045455
87,aflcid,avoid,0,0.035461
345,called,allied,0,0.454545
1580,chalman,pharma,0,0.000000
1097,cm,come,0,0.035461
121,cmng,long,0,0.035461
1239,conslderatlon,concentration,0,0.035461
858,contrlbute,contributed,0,0.035461
1006,dlstrlbutlon,redistribution,0,0.000000
487,drganlzed,organised,0,0.000000
